Jupyter notebook 
-------

This notebook illustrates the R codes to compare LASSO regression results with those results derived by Boruta algorithm used in the paper **"Data independent acquisition mass spectrometry in severe Rheumatic Heart Disease (RHD) identifies a proteomic signature showing ongoing inflammation and effectively classifying RHD cases"**

Author: **Jing Yang**

Date: **17/11/2021**

Contact: Jing.Yang@manchester.ac.uk

In [1]:
library(caret)
library(data.table)
library(tidyverse)
library(glmnet)
library(Boruta)
library(corrplot)
library(DescTools)
library(pROC)

Loading required package: ggplot2

Loading required package: lattice

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::lift()      masks caret::lift()
✖ purrr::transpose() masks data.table::transpose()

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-2

corrplot 0.90 loaded


Attaching package: ‘DescTools’


The following object is masked from ‘package:data.table’:

    %like%


The following objects 

### load log2 scaled protein expression data

In [2]:
data <- read.csv(file='Data/RHD_data_filtered.csv')
data[is.na(data)] <- 0
data$Group <- as.factor(data$Group)

### separate the data to training and testing dataset

In [3]:
set.seed(1)
trainIndex <- createDataPartition(data$Group, p=0.7, list=FALSE)
trainData <- data[trainIndex,] %>% select(-c(StollerID))
testData <- data[-trainIndex,] %>% select(-c(StollerID))


### load Boruta results

In [4]:
load(file='Data/Boruta_results_2108.RData')
result_allsample <- attStats(Boruta.allsample) %>% filter(decision %in% 'Confirmed') %>% mutate(UniProtID=rownames(.)) %>% arrange(desc(medianImp))
proteins_boruta <- result_allsample$UniProtID

In [6]:
trainData_boruta <- trainData %>% select(c(any_of(proteins_boruta), 'Group'))
testData_boruta <- testData %>% select(c(any_of(proteins_boruta), 'Group'))


In [7]:
fitControl = trainControl(method = "repeatedcv",
                          classProbs = TRUE,
                          number = 10,
                          repeats = 5, 
                          summaryFunction = twoClassSummary,
                          verboseIter = FALSE)

In [8]:
#boruta.formula <- formula(paste("Group ~ ", paste(proteins_confirmed, collapse = " + ")))
rfBoruta.fit <- train(Group ~ ., 
                      data = trainData_boruta,
                      trControl = fitControl,
                      tuneLength = 4,  # final value was mtry = 4
                      method = "rf",
                      metric = "ROC")


In [ ]:
confusionMatrix(predict(rfBoruta.fit, type='raw'), trainData$Group)

### show performance of Boruta results in training and testing data

In [9]:
confusionMatrix(trainData$Group, predict(rfBoruta.fit, newdata = trainData_boruta, type = "raw"))

confusionMatrix(testData$Group, predict(rfBoruta.fit, newdata = testData_boruta, type = "raw"))

Confusion Matrix and Statistics

          Reference
Prediction Case Control
   Case     151       0
   Control    0     161
                                     
               Accuracy : 1          
                 95% CI : (0.9882, 1)
    No Information Rate : 0.516      
    P-Value [Acc > NIR] : < 2.2e-16  
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         
                                     
            Sensitivity : 1.000      
            Specificity : 1.000      
         Pos Pred Value : 1.000      
         Neg Pred Value : 1.000      
             Prevalence : 0.484      
         Detection Rate : 0.484      
   Detection Prevalence : 0.484      
      Balanced Accuracy : 1.000      
                                     
       'Positive' Class : Case       
                                     

Confusion Matrix and Statistics

          Reference
Prediction Case Control
   Case      49      15
   Control    3      66
                                          
               Accuracy : 0.8647          
                 95% CI : (0.7946, 0.9178)
    No Information Rate : 0.609           
    P-Value [Acc > NIR] : 8.109e-11       
                                          
                  Kappa : 0.7271          
                                          
 Mcnemar's Test P-Value : 0.009522        
                                          
            Sensitivity : 0.9423          
            Specificity : 0.8148          
         Pos Pred Value : 0.7656          
         Neg Pred Value : 0.9565          
             Prevalence : 0.3910          
         Detection Rate : 0.3684          
   Detection Prevalence : 0.4812          
      Balanced Accuracy : 0.8786          
                                          
       'Positive' Class : Case            
               

### LASSO regression

In [10]:
lambdas <- 10^seq(2,-3,by=-0.1)

In [11]:
lasso_trainX <- as.matrix(trainData[,1:366])
lasso_trainy <- trainData$Group
lasso_testX <- as.matrix(testData[,1:366])
lasso_testy <- testData$Group

levels(lasso_trainy) <- c(1,0)
levels(lasso_testy) <- c(1,0)

In [12]:
lasso_reg <- cv.glmnet(lasso_trainX, lasso_trainy, alpha = 1, family = 'binomial' , lambda = lambdas, type.measure = 'deviance' , standardise=TRUE, nfolds = 4)

In [13]:
lambda_best <- lasso_reg$lambda.min

In [14]:
lasso_model <- glmnet(lasso_trainX, lasso_trainy, alpha = 1, lambda = lambda_best, family='binomial')
predictions_train <- as.factor(predict(lasso_model, s = lambda_best, newx = lasso_trainX,'class'))
#levels(predictions_train) <- levels(lasso_trainy)

predictions_test <- as.factor(predict(lasso_model, s = lambda_best, newx = lasso_testX,'class'))
#levels(predictions_test) <- levels(lasso_testy)


### show prediction performance of lasso classificatin in training and testing data

In [15]:
predictions_train <- relevel(predictions_train, ref='1')
predictions_test <- relevel(predictions_test, ref='1')


In [16]:
confusionMatrix(lasso_trainy, predictions_train)
confusionMatrix(lasso_testy, predictions_test)

Confusion Matrix and Statistics

          Reference
Prediction   1   0
         1 133  18
         0   5 156
                                          
               Accuracy : 0.9263          
                 95% CI : (0.8914, 0.9527)
    No Information Rate : 0.5577          
    P-Value [Acc > NIR] : < 2e-16         
                                          
                  Kappa : 0.852           
                                          
 Mcnemar's Test P-Value : 0.01234         
                                          
            Sensitivity : 0.9638          
            Specificity : 0.8966          
         Pos Pred Value : 0.8808          
         Neg Pred Value : 0.9689          
             Prevalence : 0.4423          
         Detection Rate : 0.4263          
   Detection Prevalence : 0.4840          
      Balanced Accuracy : 0.9302          
                                          
       'Positive' Class : 1               
                              

Confusion Matrix and Statistics

          Reference
Prediction  1  0
         1 52 12
         0  6 63
                                          
               Accuracy : 0.8647          
                 95% CI : (0.7946, 0.9178)
    No Information Rate : 0.5639          
    P-Value [Acc > NIR] : 7.888e-14       
                                          
                  Kappa : 0.728           
                                          
 Mcnemar's Test P-Value : 0.2386          
                                          
            Sensitivity : 0.8966          
            Specificity : 0.8400          
         Pos Pred Value : 0.8125          
         Neg Pred Value : 0.9130          
             Prevalence : 0.4361          
         Detection Rate : 0.3910          
   Detection Prevalence : 0.4812          
      Balanced Accuracy : 0.8683          
                                          
       'Positive' Class : 1               
                                    

In [17]:
tmp_coef <- coef(lasso_model)
result_lasso <- data.frame(name = tmp_coef@Dimnames[[1]][tmp_coef@i + 1], coefficient = tmp_coef@x) %>% pull(name)
proteins_lasso <- as.character(result_lasso[2:length(result_lasso)])

### Compare the protein signatures we get from Boruta and LASSO

In [75]:
intersected_protein <- intersect(proteins_lasso, proteins_boruta)

In [84]:
length(intersected_protein)

[1] 24

In [76]:
combined_protein <- union(proteins_lasso, proteins_boruta)

In [77]:
combined_protein_meanImp <- attStats(Boruta.allsample) %>% filter(rownames(.) %in% combined_protein) %>% arrange(desc(meanImp)) %>% 
mutate(UniProtID=rownames(.)) %>% select(c(UniProtID, meanImp))


In [78]:
combined_protein_meanImp$label <- 'Boruta'
combined_protein_meanImp$label[combined_protein_meanImp$UniProtID %in% proteins_lasso] <- 'Lasso'
combined_protein_meanImp$label[combined_protein_meanImp$UniProtID %in% intersected_protein] <- 'Lasso&Boruta'

In [79]:
protein_withname <- fread('Data/protein_withname.txt')

In [80]:
result <- left_join(combined_protein_meanImp, protein_withname) %>% select(UniProtID, ProteinName, everything()) %>% mutate_if(is.numeric, round, digits=2)

Joining, by = "UniProtID"



In [81]:
write.table(file='Data/boruta_lasso_comparison.csv', result, quote=F, row.names=F, sep=',')

In [82]:
### Show the comparison

In [83]:
print(result)

   UniProtID ProteinName meanImp        label
1     Q15848      ADIPOQ   13.67 Lasso&Boruta
2     P10643          C7   11.76 Lasso&Boruta
3     O00391       QSOX1    9.95       Boruta
4     P35858      IGFALS    9.20 Lasso&Boruta
5     P20742         PZP    8.95 Lasso&Boruta
6     P80108       GPLD1    8.45 Lasso&Boruta
7     P23142       FBLN1    7.49 Lasso&Boruta
8     P25311       AZGP1    7.00 Lasso&Boruta
9     P36955    SERPINF1    6.66       Boruta
10    P06396         GSN    6.63 Lasso&Boruta
11    P00450          CP    6.39 Lasso&Boruta
12    Q99784       OLFM1    6.06 Lasso&Boruta
13    P02743        APCS    6.04 Lasso&Boruta
14    P19320       VCAM1    5.95       Boruta
15    P02749        APOH    5.94       Boruta
16    P61626         LYZ    5.78 Lasso&Boruta
17    O75636        FCN3    5.60 Lasso&Boruta
18    P30041       PRDX6    5.44 Lasso&Boruta
19    P05546    SERPIND1    5.31       Boruta
20    P07333       CSF1R    5.26       Boruta
21    P51884         LUM    5.15 L